In [ ]:
import os
import json

import numpy as np
import pandas as pd

In [ ]:
output_dir = "/home/thanh/google_fast_or_slow/outputs_xla_default"
fold_folders = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, f)) and f.startswith("fold")]
fold_folders

In [22]:
fold_predictions_dict = {}

for folder in fold_folders:
    base_prediction = np.load(os.path.join(folder, "predictions.npy"), allow_pickle=True).item()
    base_prediction_dict = {file: prob for file, prob in zip(base_prediction["prediction_files"], base_prediction["predictions_probs"])}

    finetuning_dict = {}
    for test_file in os.listdir(os.path.join(folder, "finetuning")):
        test_file_prediction = np.load(os.path.join(folder, "finetuning", test_file, "predictions.npy"), allow_pickle=True).item()
        for file, prob in zip(test_file_prediction["prediction_files"], test_file_prediction["predictions_probs"]):
            finetuning_dict[file] = prob

    final_dict = {}
    for file, prob in base_prediction_dict.items():
        final_dict[file] = prob
    for file, prob in finetuning_dict.items():
        final_dict[file] = prob
    
    fold_predictions_dict[os.path.basename(folder)] = final_dict

In [32]:
from collections import defaultdict

mean_predictions_dict = defaultdict(list)

for file in fold_predictions_dict["fold_0"].keys():
    for key in fold_predictions_dict.keys():
        mean_predictions_dict[file].append(fold_predictions_dict[key][file])

mean_predictions_dict = {file: np.mean(np.array(probs), axis=0) for file, probs in mean_predictions_dict.items()}
# get argsort
mean_predictions_dict = {file: np.argsort(probs) for file, probs in mean_predictions_dict.items()}

df = pd.DataFrame.from_dict({
    "ID": mean_predictions_dict.keys(),
    "TopConfigs": mean_predictions_dict.values()
})
df["TopConfigs"] = df["TopConfigs"].apply(lambda x: ";".join([str(i) for i in x]))
df

,ID,TopConfigs
0,layout:xla:default:05ae41e26dd3c4c06390371a042...,481;575;988;295;197;832;302;450;770;912;67;908...
1,layout:xla:default:3e7156ac468dfb75cf5c9615e1e...,263;903;324;111;97;376;131;52;530;893;882;153;...
2,layout:xla:default:5335ed13823b0a518ee3c79ba44...,631;296;710;978;664;398;454;179;196;746;99;384...
3,layout:xla:default:937ee0eb0d5d6151b7b8252933b...,149;272;126;429;605;128;337;296;414;495;29;789...
4,layout:xla:default:cd708819d3f5103afd6460b15e7...,502;412;80;239;31;308;155;124;87;89;57;732;190...
5,layout:xla:default:db59a991b7c607634f13570d52c...,179;353;730;999;712;546;837;48;249;68;871;296;...
6,layout:xla:default:e8a3a1401b5e79f66d7037e424f...,996;437;210;487;238;114;388;165;9;366;221;413;...
7,layout:xla:default:fbaa8bb6a1aed9988281085c910...,681;539;401;161;425;775;986;872;824;474;211;50...
